In [2]:
from kedro.io import DataCatalog
from kedro.extras.datasets.pandas import CSVDataSet
import h5py
import glob
import pandas as pd
import numpy as np

In [6]:
data_catalog = CSVDataSet(filepath="../data/01_raw/catalog.csv",
                        load_args={"sep": ';'})
io = DataCatalog(data_sets={'data_catalog': data_catalog})

df = io.load("data_catalog")
df

,cell_line,compound,file_path,note
0,D5,NaN,210315/D5/2599,NaN
1,PKP2-KO,NaN,210802/KO,NaN


In [7]:
df.loc[0,'file_path']

'210315/D5/2599'

In [16]:
# list all files in the directory
df2 = pd.DataFrame(columns=df.columns.to_list() + ['file_path_full'])
base_dir = "/mnt/z/recordings/Mea1k/leejih/"

for i in range(2):#len(df)):
    rec_files = glob.glob(base_dir+df.loc[i,'file_path']+'/*.h5')
    try:
        tmp = pd.concat([df.loc[[i]]]*len(rec_files), ignore_index=True)
        tmp["file_path_full"] = rec_files
        df2 = df2.merge(tmp, how='outer')
    except Exception as e:
        print(e)
df2

,cell_line,compound,file_path,note,file_path_full
0,D5,NaN,210315/D5/2599,NaN,/mnt/z/recordings/Mea1k/leejih/210315/D5/2599/...
1,D5,NaN,210315/D5/2599,NaN,/mnt/z/recordings/Mea1k/leejih/210315/D5/2599/...
2,D5,NaN,210315/D5/2599,NaN,/mnt/z/recordings/Mea1k/leejih/210315/D5/2599/...
3,D5,NaN,210315/D5/2599,NaN,/mnt/z/recordings/Mea1k/leejih/210315/D5/2599/...
4,D5,NaN,210315/D5/2599,NaN,/mnt/z/recordings/Mea1k/leejih/210315/D5/2599/...
5,D5,NaN,210315/D5/2599,NaN,/mnt/z/recordings/Mea1k/leejih/210315/D5/2599/...
6,PKP2-WT,NaN,220908/WT,NaN,/mnt/z/recordings/Mea1k/leejih/220908/WT/3994....
7,PKP2-WT,NaN,220908/WT,NaN,/mnt/z/recordings/Mea1k/leejih/220908/WT/3997....
8,PKP2-WT,NaN,220908/WT,NaN,/mnt/z/recordings/Mea1k/leejih/220908/WT/4002....
9,PKP2-WT,NaN,220908/WT,NaN,/mnt/z/recordings/Mea1k/leejih/220908/WT/4003....


In [15]:
base_dir = "/mnt/z/recordings/Mea1k/leejih/"
rec_files = glob.glob(base_dir+df.loc[2,'file_path']+'/*.h5')
rec_files
try:
    tmp = pd.concat([df.loc[[0]]]*len(rec_files), ignore_index=True)
    tmp["file_path_full"] = rec_files
    tmp
except Exception as e:
    print(e)

No objects to concatenate


In [1]:
# Write a yaml file
import yaml

data = dict(n_files = 5)

def write_yaml(data):
    """A function to write YAML file."""
    with open('toyaml.yml', 'w') as f:
        yaml.dump(data, f)

write_yaml(data)

In [3]:
filepath = "../data/01_raw/highamp.raw.h5"

obj = h5py.File(filepath, mode='r')
lsb = obj['settings']['lsb'][0] * 1e6

gain = (obj['settings']['gain'][0]).astype(int)
if 'lsb' in obj['settings']:
    lsb = obj['settings']['lsb'][0] * 1e6
else:
    lsb = 3.3 / (1024 * gain) * 1e6

mapping = obj.get('mapping')
channels = np.array(mapping['channel'])
electrodes = np.array(mapping['electrode'])

# remove unused channels
routed_idxs = np.where(electrodes > -1)[0]

channel_ids = list(channels[routed_idxs])
electrode_ids = list(electrodes[routed_idxs])
num_channels = len(electrode_ids)
num_frames = len(obj.get('sig')[0,:])
x_locs = mapping['x']
y_locs = mapping['y']

In [17]:
start_frame = 0
length = num_frames
signals = (obj.get('sig')[np.array(channel_ids),0:min(int(10.0),int(5.0))] * lsb).astype('float32') 

In [1]:
from joblib import Parallel, delayed

In [22]:
def tes(x):
    return x*10, x**2

res = Parallel(n_jobs=4, backend='multiprocessing')(delayed(tes)(i) for i in range(8))
print(res)

[(0, 0), (10, 1), (20, 4), (30, 9), (40, 16), (50, 25), (60, 36), (70, 49)]
